In [1]:
import boto3
import json
import botocore
from IPython.display import display, Markdown
import time

Langgraph is a framework to build llm agents, it provide lot of tools like way conencting to deployed models, options to create agents with different optimization techniques in terms of memory, human in loop etc.,

In this notebook I am trying to use aws bedrock completely to communicate with the models in aws bedrock, in this way it provide me access to different aws services that can be utilized

Initially it will be simple implemention where I should be able to query the llm in bedrock and later will try to incorporate more features like langchain prompt techniqes etc to get the better response from models

In [2]:
# Initialize the bedrock client

session = boto3.session.Session()
region =  session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)

In [3]:
bedrock

In [4]:
#defining the different available models

MODELS = {
  "Claude 3.7 Sonnet": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
  "Claude 3.5 Sonnet": "us.anthropic.claude-3-5-sonnet-20240620-v1:0",
  "Claude 3.5 Haiku": "us.anthropic.claude-3-5-haiku-20241022-v1:0",
  "Amazon Nova Pro": "us.amazon.nova-pro-v1:0",
  "Amazon Nova Micro": "us.amazon.nova-micro-v1:0",
  "DeepSeek-R1": "us.deepseek.r1-v1:0",
  "Meta Llama 3.1 70B Instruct": "us.meta.llama3-1-70b-instruct-v1:0"
}

In [5]:
#utility function to display the model response
def display_response(response, model_name = None):
    if model_name:
      display(Markdown(f"### Response from **{model_name}**:\n\n"))
    display(Markdown(response))
    print("\n"+"-"*80 + "\n")

In [6]:
# Text summarization using bedrock invoke api
# otehr ways to use the API is converse API which is recommended

text_to_summarize = """
  AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
  a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
  Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
  democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
  for text and images—including Amazons Titan FMs, which consist of two new LLMs we're also announcing \
  today—through a scalable, reliable, and secure AWS managed service. With Bedrock's serverless experience, \
  customers can easily find the right model for what they're trying to get done, get started quickly, privately \
  customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
  tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
  with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
  """

In [7]:
prompt = f"""summairize the information in the following text"
<text>
{text_to_summarize}
</text>
"""

In [12]:
claude_body = json.dumps({
  "anthropic_version": "bedrock-2023-05-31",
  "max_tokens": 1000,
  "temperature": 0.5,
  "top_p": 0.9,
  "messages": [
    {
      "role": "user",
      "content": [{"type": "text", "text": prompt}]
    }
  ]
})

In [13]:
try:
    response = bedrock.invoke_model(
        modelId=MODELS["Claude 3.7 Sonnet"],
        body=claude_body,
        accept="application/json",
        contentType="application/json"
    )
    response_body = json.loads(response.get('body').read())
    
    # Extract and display the response text
    claude_summary = response_body["content"][0]["text"]
    display_response(claude_summary, "Claude 3.7 Sonnet (Invoke Model API)")
    
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Message']}\
            \nTo troubleshoot this issue please refer to the following resources.\
            \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
            \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
    else:
        raise error

### Response from **Claude 3.7 Sonnet (Invoke Model API)**:



# Amazon Bedrock Announcement Summary

AWS has announced Amazon Bedrock, a new service that provides API access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon itself. Key features include:

- Easy access to powerful text and image foundation models, including Amazon's new Titan LLMs
- Serverless experience with no infrastructure management required
- Ability for customers to privately customize models with their own data
- Integration with existing AWS tools like SageMaker ML features
- Support for testing different models and managing FMs at scale

The service aims to democratize access to generative AI by making it easier for all builders to develop and scale AI-based applications.


--------------------------------------------------------------------------------



Invoke api genrally ahs direct access to the models but still has many limitations

1. Input/Output format is different
2. no built-in support for multi conversations
3. custom hadling for different model capabilities

In below I am implementing the same summarization using converse api


In [14]:
converse_request = {
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": f"Please provide a concise summary of the following text in 2-3 sentences. Text to summarize: {text_to_summarize}"
        }
      ]
    }
  ],
  "inferenceConfig": {
    "maxTokens": 1000,
    "temperature": 0.5,
    "topP": 0.9
  }
}

In [15]:
try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )
    
    # Extract the model's response
    claude_converse_response = response["output"]["message"]["content"][0]["text"]
    display_response(claude_converse_response, "Claude 3.7 Sonnet (Converse API)")
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AccessDeniedException':
        print(f"\x1b[41m{error.response['Error']['Code']}: {error.response['Error']['Message']}\x1b[0m")
        print("Please ensure you have the necessary permissions for Amazon Bedrock.")
    else:
        raise error

### Response from **Claude 3.7 Sonnet (Converse API)**:



Amazon Bedrock is a new service that provides API access to foundation models from various AI companies, including Amazon's own Titan models, enabling developers to build generative AI applications. It offers a serverless experience where customers can select, customize, and deploy models without managing infrastructure, making advanced AI capabilities more accessible to all builders.


--------------------------------------------------------------------------------



Converse api supports the following parameters while generating a request
1. modelId - name of the model
2. messages - conversational messages/history
3. toolsConfig - list of the tools that model can trigger to generate the output
4. inference config

Documentation - https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html

In [16]:
multi_turn_messages = [
    {
        "role": "user",
        "content": [{"text": f"Please summarize this text: {text_to_summarize}"}]
    },
    {
        "role": "assistant",
        "content": [{"text": claude_converse_response}]
    },
    {
        "role": "user",
        "content": [{"text": "Can you make this summary even shorter, just 1 sentence?"}]
    }
]

In [17]:
try:
    response = bedrock.converse(
        modelId=MODELS["Claude 3.7 Sonnet"],
        messages=multi_turn_messages,
        inferenceConfig={"temperature": 0.2, "maxTokens": 500}
    )
    
    # Extract the model's response using the correct structure
    follow_up_response = response["output"]["message"]["content"][0]["text"]
    display_response(follow_up_response, "Claude 3.7 Sonnet (Multi-turn conversation)")
    
except Exception as e:
    print(f"Error: {str(e)}")

### Response from **Claude 3.7 Sonnet (Multi-turn conversation)**:



Amazon Bedrock is a new serverless service that provides API access to foundation models from multiple AI companies, allowing developers to easily build, customize, and deploy generative AI applications without managing infrastructure.


--------------------------------------------------------------------------------



Till now we saw invoking the model directly using bedrock.invoke, as it has own limitations started implementing converse api but still it has some limitations majorly in converse api we need to wait until the whole response from model gets generated which is kind of not user friendly

So bedrock supports conversestream api which helps in receiving the content as its being generated, it supports streaming so long generation response can be received incrementally

In [18]:
def stream_converse(model_id, messages, inference_config=None):
    if inference_config is None:
        inference_config = {}
    
    print("Streaming response (chunks will appear as they are received):\n")
    print("-" * 80)
    
    full_response = ""
    
    try:
        response = bedrock.converse_stream(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config
        )
        response_stream = response.get('stream')
        if response_stream:
            for event in response_stream:

                if 'messageStart' in event:
                    print(f"\nRole: {event['messageStart']['role']}")

                if 'contentBlockDelta' in event:
                    print(event['contentBlockDelta']['delta']['text'], end="")

                if 'messageStop' in event:
                    print(f"\nStop reason: {event['messageStop']['stopReason']}")

                if 'metadata' in event:
                    metadata = event['metadata']
                    if 'usage' in metadata:
                        print("\nToken usage")
                        print(f"Input tokens: {metadata['usage']['inputTokens']}")
                        print(
                            f":Output tokens: {metadata['usage']['outputTokens']}")
                        print(f":Total tokens: {metadata['usage']['totalTokens']}")
                    if 'metrics' in event['metadata']:
                        print(
                            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")

                
            print("\n" + "-" * 80)
        return full_response
    
    except Exception as e:
        print(f"Error in streaming: {str(e)}")
        return None

In [19]:
streaming_request = [
    {
        "role": "user",
        "content": [
            {
                "text": f"""Please provide a detailed summary of the following text, explaining its key points and implications:
                
                {text_to_summarize}
                
                Make your summary comprehensive but clear.
                """
            }
        ]
    }
]

In [20]:
streamed_response = stream_converse(
    MODELS["Claude 3.7 Sonnet"], 
    streaming_request, 
    inference_config={"temperature": 0.4, "maxTokens": 1000}
)

Streaming response (chunks will appear as they are received):

--------------------------------------------------------------------------------

Role: assistant
# Summary: Amazon Bedrock Announcement

## Key Points

Amazon Bedrock is a new AWS service that provides API access to foundation models (FMs) from multiple AI providers, including:
- AI21 Labs
- Anthropic
- Stability AI
- Amazon's own Titan models (newly announced)

The service offers both text and image generation capabilities through a serverless, managed platform.

## Core Features and Benefits

1. **Democratized Access**: Makes powerful foundation models accessible to all developers without requiring specialized expertise.

2. **Serverless Architecture**: Eliminates infrastructure management needs, allowing customers to focus on application development.

3. **Model Selection**: Provides a range of models to choose from based on specific use case requirements.

4. **Private Customization**: Allows customers to customize fou